# Accessing ACME
### Ray Smith

In [21]:
# Selenium Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# Data Handling Imports
import numpy as np
import pandas as pd

# Datetime Imports
import datetime

In [22]:
# Settings
url = "https://acme.wisc.edu/tools/schedule/schedule.php"

login_secret = "login.secret"
duo_secret = "duo.secret"

options = Options()
# options.headless = True

In [23]:
# Get Secrets
with open(login_secret) as f:
    user = f.readline()[:-1]
    password = f.readline()

In [24]:
# Date conversion functions
# String form = date=YYYY-MM-DD

# URL String to Date
def string_to_date(string):
    parts = string.split('-')
    year = int(parts[0])
    month = int(parts[1])
    day = int(parts[2])
    date = datetime.date(year=year, month=month, day=day)
    return date

# Date to URL String
def date_to_string(date):
    year = date.year
    month = date.month
    day = date.day
    return str(year) + '-' + str(month) + '-' + str(day)

In [25]:
# Login to ACME
def acme_login():
    browser = webdriver.Chrome(options=options)
    browser.get(url)

    # Fill in username
    user_field = browser.find_element_by_id('j_username')
    user_field.send_keys(user)

    # Fill in password
    pass_field = browser.find_element_by_id('j_password')
    pass_field.send_keys(password)

    # Click login
    login_button = browser.find_element_by_name('_eventId_proceed')
    login_button.click()
    
    # Return ACME pointed browser
    return browser

In [26]:
# Get Schedule Table by Date (as string)
def get_schedule_by_date(browser, date):
    # Access page for specified date
    new_url = url + '?date=' + date
    browser.get(new_url)
    
    # Get table headers
    header_html = browser.find_element_by_xpath("//thead[1]/tr").get_attribute('innerHTML')
    headers = []
    header_text = header_html.split(" ")
    for line in header_text:
        if line.startswith('id='):
            parts = line.split('"')
            header = parts[1]
            parts = header.split('_')
            headers.append(parts[1])
    headers.append("total")
    
    # Get table
    table = browser.find_element_by_id('sch_table_verticle')
    table_html = table.get_attribute('outerHTML')
    df = pd.read_html(table_html)[0]
    df.drop(df.tail(2).index,inplace=True)
    
    # Set headers of dataframe
    columns = df.columns
    new_names = {}
    for i in range(len(columns)):
        new_names[columns[i]] = headers[i]
    df = df.rename(columns=new_names)
    df = df.set_index('time')
    df = df.drop(columns=['total'])
    
    # Split up agents
    for col in df:
        data = df[col]
        new_data = []
        for row in data:
            if type(row) != str:
                new_data.append(None)
            else:
                agents = [row[i:i+4] for i in range(0, len(row), 4)]
                new_data.append(agents)
        df[col] = new_data
    
    return df

In [27]:
# Login
browser = acme_login()

In [28]:
# Access certain date
date = '2020-1-15'
table= get_schedule_by_date(browser, date)
table

,hdqa,techstore,phones,chatemail,hdqaremote,phonesremote,hdl1project,walkinlead,walkincounter,walkinsd,repair
time,,,,,,,,,,,
7:00AM,[REMI],[CTER],[HUNT],None,None,None,None,None,None,None,[WRIT]
7:30AM,[REMI],[CTER],"[HUNT, SPEK]",None,None,None,None,None,[DPAY],[AWEN],[WRIT]
8:00AM,[REMI],[CTER],[SPEK],[HUNT],None,None,None,None,[DPAY],[AWEN],[WRIT]
8:30AM,[REMI],[CTER],"[SPEK, AGOE]",[HUNT],None,None,None,[MIMO],[DPAY],[AWEN],[WRIT]
9:00AM,[JMEN],[LKRE],"[CTER, EWJE, SPEK, BANT, AGOE, JOHA, REMI]",[HUNT],None,None,[ELTR],[MIMO],"[JWHY, DPAY]",[AWEN],[WRIT]
9:30AM,[JMEN],[LKRE],"[CTER, EWJE, SPEK, BANT, AGOE, JOHA, REMI]",[HUNT],None,None,[ELTR],[MIMO],"[JWHY, DPAY]",[AWEN],[WRIT]
10:00AM,[JMEN],[LKRE],"[CTER, EWJE, SPEK, BANT, AGOE, JOHA, SWAL]",[HUNT],None,None,"[ELTR, REMI]",[MIMO],"[JWHY, DPAY]",[AWEN],"[MDUF, WRIT]"
10:30AM,[JMEN],[LKRE],"[CTER, EWJE, SPEK, BANT, AGOE, JOHA, SWAL]",[HUNT],None,None,"[ELTR, REMI]",[MIMO],[JWHY],"[AWEN, TDHA]","[MDUF, WRIT, DPAY]"
11:00AM,[JMEN],[LKRE],"[HUNT, CTER, EWJE, SPEK, AGOE, JOHA, SWAL]",[EMAC],None,None,[REMI],[MIMO],[ADWI],"[JESO, AWEN, TDHA]","[MDUF, WRIT, DPAY]"


In [29]:
# Close browser
browser.close()

In [39]:
# ACME Class
class ACME:
    
    #===================================================================
    # Settings
    #===================================================================
    
    url = "https://acme.wisc.edu/tools/schedule/schedule.php"
    login_secret = "login.secret"
    
    #===================================================================
    # Setup
    #===================================================================
    
    def __init__(self, headless = False):
        # Set Mode
        self.options = Options()
        self.options.headless = headless
        
        # Get Secrets
        with open(login_secret) as f:
            self.user = f.readline()[:-1]
            self.password = f.readline()
    
    # Login to ACME
    def login(self):
        browser = webdriver.Chrome(options=self.options)
        browser.get(url)

        # Fill in username
        user_field = browser.find_element_by_id('j_username')
        user_field.send_keys(self.user)

        # Fill in password
        pass_field = browser.find_element_by_id('j_password')
        pass_field.send_keys(self.password)

        # Click login
        login_button = browser.find_element_by_name('_eventId_proceed')
        login_button.click()

        # Return ACME pointed browser
        self.browser = browser
        return self.browser
    
    #===================================================================
    # ACME Functionality
    #===================================================================
    
    # Get Schedule Table by Date (as string)
    def get_schedule_by_date(self, date):
        # Access page for specified date
        new_url = url + '?date=' + date
        browser = self.browser
        browser.get(new_url)

        # Get table headers
        header_html = browser.find_element_by_xpath("//thead[1]/tr").get_attribute('innerHTML')
        headers = []
        header_text = header_html.split(" ")
        for line in header_text:
            if line.startswith('id='):
                parts = line.split('"')
                header = parts[1]
                parts = header.split('_')
                headers.append(parts[1])
        headers.append("total")

        # Get table
        table = browser.find_element_by_id('sch_table_verticle')
        table_html = table.get_attribute('outerHTML')
        df = pd.read_html(table_html)[0]
        df.drop(df.tail(2).index,inplace=True)

        # Set headers of dataframe
        columns = df.columns
        new_names = {}
        for i in range(len(columns)):
            new_names[columns[i]] = headers[i]
        df = df.rename(columns=new_names)
        df = df.set_index('time')
        df = df.drop(columns=['total'])

        # Split up agents
        for col in df:
            data = df[col]
            new_data = []
            for row in data:
                if type(row) != str:
                    new_data.append(None)
                else:
                    agents = [row[i:i+4] for i in range(0, len(row), 4)]
                    new_data.append(agents)
            df[col] = new_data

        return df
    
    # Close the browser
    def close(self):
        self.browser.close()
    
    #===================================================================
    # Utilities
    #===================================================================
    
    # Date conversion functions
    # String form = date=YYYY-MM-DD

    # URL String to Date
    def string_to_date(self, string):
        parts = string.split('-')
        year = int(parts[0])
        month = int(parts[1])
        day = int(parts[2])
        date = datetime.date(year=year, month=month, day=day)
        return date

    # Date to URL String
    def date_to_string(self, date):
        year = date.year
        month = date.month
        day = date.day
        return str(year) + '-' + str(month) + '-' + str(day)

In [49]:
acme = ACME()
acme.login()

<selenium.webdriver.chrome.webdriver.WebDriver (session="aafcaf3db62535b3dc4c11ec13d9016e")>

In [50]:
date = '2020-1-15'
table = acme.get_schedule_by_date(date)
table

,hdqa,techstore,phones,chatemail,hdqaremote,phonesremote,hdl1project,walkinlead,walkincounter,walkinsd,repair
time,,,,,,,,,,,
7:00AM,[REMI],[CTER],[HUNT],None,None,None,None,None,None,None,[WRIT]
7:30AM,[REMI],[CTER],"[HUNT, SPEK]",None,None,None,None,None,[DPAY],[AWEN],[WRIT]
8:00AM,[REMI],[CTER],[SPEK],[HUNT],None,None,None,None,[DPAY],[AWEN],[WRIT]
8:30AM,[REMI],[CTER],"[SPEK, AGOE]",[HUNT],None,None,None,[MIMO],[DPAY],[AWEN],[WRIT]
9:00AM,[JMEN],[LKRE],"[CTER, EWJE, SPEK, BANT, AGOE, JOHA, REMI]",[HUNT],None,None,[ELTR],[MIMO],"[JWHY, DPAY]",[AWEN],[WRIT]
9:30AM,[JMEN],[LKRE],"[CTER, EWJE, SPEK, BANT, AGOE, JOHA, REMI]",[HUNT],None,None,[ELTR],[MIMO],"[JWHY, DPAY]",[AWEN],[WRIT]
10:00AM,[JMEN],[LKRE],"[CTER, EWJE, SPEK, BANT, AGOE, JOHA, SWAL]",[HUNT],None,None,"[ELTR, REMI]",[MIMO],"[JWHY, DPAY]",[AWEN],"[MDUF, WRIT]"
10:30AM,[JMEN],[LKRE],"[CTER, EWJE, SPEK, BANT, AGOE, JOHA, SWAL]",[HUNT],None,None,"[ELTR, REMI]",[MIMO],[JWHY],"[AWEN, TDHA]","[MDUF, WRIT, DPAY]"
11:00AM,[JMEN],[LKRE],"[HUNT, CTER, EWJE, SPEK, AGOE, JOHA, SWAL]",[EMAC],None,None,[REMI],[MIMO],[ADWI],"[JESO, AWEN, TDHA]","[MDUF, WRIT, DPAY]"


In [51]:
acme.browser.close()